![](SARSA.png)

- Estado-Acción-Recompensa-Estado-Acción

- Aprender la mejor acción en cada estado.

- SARSA toma en cuenta el esrado actual, como la acción actual para predecir la próxima acción y su recompensa.
- Aprendizaje On policy.

## ¿Qué significa esta fórmula?

Es la fórmula de **actualización de Q en SARSA**. A diferencia de Q-learning, que usa el **mejor valor futuro posible**, SARSA actualiza el valor **basado en la acción realmente tomada** en el siguiente paso.

## Descripción de cada término:

| Símbolo     | Significado                                                                                                                       |
| ----------- | --------------------------------------------------------------------------------------------------------------------------------- |
| `Q(s, a)`   | Valor actual de tomar la acción `a` en el estado `s`.                                                                             |
| `α` (alfa)  | Tasa de aprendizaje (cuánto se ajusta el valor Q actual).                                                                         |
| `r`         | Recompensa inmediata obtenida al hacer `a` desde `s`.                                                                             |
| `γ` (gamma) | Factor de descuento (cuánto importa el futuro).                                                                                   |
| `Q(s', a')` | Valor esperado de tomar la **acción elegida** `a'` en el **nuevo estado** `s'`. **Aquí está la diferencia clave con Q-learning**. |

---

### ¿Qué hace SARSA?

> Actualiza la tabla `Q` usando el valor de **la acción que realmente tomó el agente**, no la mejor posible.

Por eso su nombre:
**S → A → R → S' → A'**
(state, action, reward, next state, next action)

## Comparación con Q-learning

| Algoritmo  | Usa en la actualización                   |
| ---------- | ----------------------------------------- |
| Q-learning | `max_a' Q(s', a')` (mejor acción posible) |
| SARSA      | `Q(s', a')` (acción realmente tomada)     |

### ¿Por qué importa esto?

* **SARSA es más conservador**: aprende de lo que el agente *hizo*, no de lo que *hubiera sido mejor*.
* Esto puede ser útil en **entornos donde el riesgo importa** (por ejemplo, evitar colisiones en robótica).

In [85]:
import numpy as np
import random
import matplotlib.pyplot as plt

In [86]:
dimensiones = (4, 4)
estado_inicial = (0, 0)
estado_final = (3, 3)
acciones = [(0,-1), (0,1), (-1,0), (1,0)]
acciones_simbolos = ['^', 'v', '<', '>']

In [87]:
num_estados = dimensiones[0] * dimensiones[1]
num_acciones = len(acciones)

In [88]:
Q = np.zeros((num_estados, num_acciones))
Q

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

- Alpha: 
  - Factor de la taza de aprendizaje. 
  - Cuánto se actualiza el valor Q en cada paso. 
  - Valor bajo es más lento pero más seguro
- Gamma:
  - Factor de descuento. 
  - Determinar la importancia de las recompensas que va a obtener en el futuro.
  - Cercano a 1 hace que las recompensas futuras sean casi tan importantes como las inmediatas, haciendo que el agente considera consecuencias a largo plazo de sus acciones.
  - Valos más bajo, al agente va a valorar más las consecuencias inmediatas.
- Epsilon:
  - Sirve para que el agente no repita siempre las mismas decisiones.
  - Se define la probabilidad de que el agente tome una acción aleatoria en lugar de que el agente tome una acción conocida en la tabla Q.
  - Permite que el agente explore el entorno.
- Episodios:
  - Define el número total de veces que se va a repetir el proceso de entrenamiento.
  - Empieza con el agente en el estado inicial y termina cuando alcance el objetivo.

In [89]:
alpha = 0.1
gamma = 0.99
epsilon = 0.2
episodios = 1000

In [90]:
def estado_a_indice(estado):
    return estado[0] * dimensiones[1] + estado[1]

In [91]:
def elegir_accion(estado):
    if random.uniform(0,1) < epsilon:
        return random.randint(0, num_acciones-1)
    else:
        return np.argmax(Q[estado_a_indice(estado)])

In [92]:
def aplicar_accion(estado, accion_idx):
    
    accion = acciones[accion_idx]
    
    nuevo_estado = tuple(np.add(estado, accion) % np.array(dimensiones))
    
    if nuevo_estado == estado_final:
        recompensa = 1
    else:
        recompensa = -1
    
    return nuevo_estado, recompensa, nuevo_estado == estado_final

In [93]:
for episodio in range(episodios):

    estado = estado_inicial
    terminado = False
    accion_idx = elegir_accion(estado)

    while not terminado:

        nuevo_estado, recompensa, terminado = aplicar_accion(estado, accion_idx)

        nuevo_accion_idx = elegir_accion(nuevo_estado)

        indice = estado_a_indice(estado)

        Q[indice, accion_idx] += alpha * (recompensa + gamma * Q[estado_a_indice(nuevo_estado), nuevo_accion_idx] - Q[indice, accion_idx])

        estado, accion_idx = nuevo_estado, nuevo_accion_idx

In [94]:
politica_simbolos = np.empty(dimensiones, dtype="<U2")
politica_simbolos

array([['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', '']], dtype='<U2')

In [95]:
for i in range(dimensiones[0]):
    for j in range(dimensiones[1]):
        estado = (i, j)
        accion_idx = np.argmax(Q[estado_a_indice(estado)])
        politica_simbolos[i, j] = acciones_simbolos[accion_idx]
politica_simbolos

array([['^', 'v', '<', '<'],
       ['>', '>', '<', '<'],
       ['>', 'v', '>', '>'],
       ['^', 'v', 'v', '^']], dtype='<U2')

In [130]:
import numpy as np
import random
import matplotlib.pyplot as plt

# Definición del entorno: una cuadrícula 4x4
dimensiones = (4, 4)
estado_inicial = (0, 0)
estado_final = (3, 3)

# Acciones posibles: izquierda, derecha, arriba, abajo
acciones = [(0, -1), (0, 1), (-1, 0), (1, 0)]
acciones_simbolos = ['^', 'v', '<', '>']

# Número total de estados y acciones
num_estados = dimensiones[0] * dimensiones[1]
num_acciones = len(acciones)

# Inicialización de la tabla Q con ceros
Q = np.zeros((num_estados, num_acciones))

# Parámetros de aprendizaje
alpha = 0.1      # tasa de aprendizaje
gamma = 0.99     # factor de descuento
epsilon = 0.2    # probabilidad de explorar
episodios = 1000 # número total de episodios de entrenamiento

# Convierte un estado (fila, columna) a un índice de la tabla Q
def estado_a_indice(estado):
    return estado[0] * dimensiones[1] + estado[1]

# Elige una acción con política ε-greedy (exploración/explotación)
def elegir_accion(estado):
    if random.uniform(0, 1) < epsilon:
        return random.randint(0, num_acciones - 1)  # acción aleatoria
    else:
        return np.argmax(Q[estado_a_indice(estado)])  # mejor acción conocida

# Aplica una acción al estado actual y devuelve:
# nuevo estado, recompensa obtenida y si el episodio terminó
def aplicar_accion(estado, accion_idx):
    accion = acciones[accion_idx]
    nuevo_estado = tuple(np.add(estado, accion) % np.array(dimensiones))  # wrap-around

    # Recompensa +1 si se alcanza el estado objetivo, -1 en otro caso
    if nuevo_estado == estado_final:
        recompensa = 1
    else:
        recompensa = -1

    return nuevo_estado, recompensa, nuevo_estado == estado_final

# Entrenamiento usando SARSA
for episodio in range(episodios):
    estado = estado_inicial
    terminado = False
    accion_idx = elegir_accion(estado)  # elegir acción inicial

    while not terminado:
        # Aplicar la acción elegida
        nuevo_estado, recompensa, terminado = aplicar_accion(estado, accion_idx)

        # Elegir siguiente acción (SARSA aprende de la acción realmente tomada)
        nuevo_accion_idx = elegir_accion(nuevo_estado)

        # Obtener el índice del estado actual para acceder a la Q-table
        indice = estado_a_indice(estado)

        # Actualizar Q usando la fórmula de SARSA
        Q[indice, accion_idx] += alpha * (
            recompensa + gamma * Q[estado_a_indice(nuevo_estado), nuevo_accion_idx]
            - Q[indice, accion_idx]
        )

        # Avanzar al nuevo estado y acción
        estado, accion_idx = nuevo_estado, nuevo_accion_idx


def mostrar_politica_sarsa():
    print("\nPolítica aprendida con SARSA:\n")
    for fila in range(dimensiones[0]):
        linea = ""
        for col in range(dimensiones[1]):
            estado = (fila, col)
            if estado == estado_final:
                linea += " 🎯 "  # objetivo
            else:
                idx = estado_a_indice(estado)
                mejor_accion = np.argmax(Q[idx])
                linea += f" {acciones_simbolos[mejor_accion]}  "
        print(linea)

mostrar_politica_sarsa()



Política aprendida con SARSA:

 ^   >   <   <  
 ^   ^   >   >  
 >   >   >   >  
 ^   v   v   🎯 
